In [5]:
#!pip install -U twitter-client-py
import streamlit as st

In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from dotenv import load_dotenv
import os
import requests
import pprint as ppp
from twitter_client_py import TwitterClient, AsyncTwitterClient

In [7]:
# Q de consultas que vamos (limite mensual = 1.000)
api_key = os.getenv('2e2c904e0cmshdbad92d97808688p1e798ajsna8b04a4dd68a')
client = AsyncTwitterClient(api_key=api_key, timeout=20, verbose=True)
rate_limit = client.rate_limit
print(f"Limit: {rate_limit.limit}, Remaining requests: {rate_limit.remaining}, Reset time: {rate_limit.reset}")

Limit: 0, Remaining requests: 0, Reset time: 0


La proxima celda corresponde al codigo para trabajar en una API con cantidad de consultas limitadas. Se me ocurre que lo que habria que hacer, para evitar bloqueos (o exceder las cuotas de consulta), seria tener varios user con varias apis, e ir tomando de a uno por vez.

In [8]:
url_tweets_search_api_01 = "https://twitter-x.p.rapidapi.com/search/"

user_search_phrase = 'Madrid'

querystring = {"query":user_search_phrase,"section":'latest',"limit":"20", 'lang':'en'} 			# "section" predeterminado sin posibilidad de cambiar por el cliente (para traer el sentimiento en tiempo real)
# el parametro "lang" no funciona, hay que resolverlo en el EDA (PROBAR CON LA API XCOM)

headers = {
	"x-rapidapi-key": "2e2c904e0cmshdbad92d97808688p1e798ajsna8b04a4dd68a",
	"x-rapidapi-host": "twitter-x.p.rapidapi.com"
}

response_api_01 = requests.get(url_tweets_search_api_01, headers=headers, params=querystring)
entries_api_01 = response_api_01.json()['data']['search_by_raw_query']['search_timeline']['timeline']['instructions'][0]['entries']

def clean_entries_with_dates(list_of_elem):
    clean_data = []
    for element in list_of_elem:
        content = element.get('content',[])                                                                             # para extraer el contenido de 'content'
        item_content = content.get('itemContent',{})                                                                    # para extraer el contenido de 'itemContent'
        if 'tweet_results' not in item_content or 'result' not in item_content['tweet_results'] : continue              # que siga de largo con este elemento si no tiene 'tweet_result' o 'result'
        result = item_content['tweet_results']['result']                                                                # para extraer el contenido de 'tweet_results'
        if 'legacy' not in result or 'full_text' not in result['legacy']: continue                                      # que siga de largo con este elemento si no tiene 'full_text' o 'legacy'
        full_text = result['legacy']['full_text']                                                                       # para extraer el contenido de 'full_text'
        post_date = result['legacy']['created_at']
        clean_data.append((post_date, full_text))
    return clean_data

aux_prueba = clean_entries_with_dates(entries_api_01)
aux_prueba = pd.DataFrame(aux_prueba)
aux_prueba																						# Resultado final

,0,1
0,Sat Oct 12 11:24:21 +0000 2024,No hay mucha discusión \nLamine\nA Vini lo veo...
1,Sat Oct 12 11:24:21 +0000 2024,yes i'm still salty i can't attend music bank ...
2,Fri Oct 11 09:32:00 +0000 2024,Vielä ehdit! Liittyminen ja 1. treenikk 10 eur...
3,Sat Oct 12 11:24:20 +0000 2024,@sxftmdx fuersas nosotros hemos llegado casi 2...
4,Sat Oct 12 11:24:19 +0000 2024,@HolaSoyMarisa4 En cuanto empiezas a insultarm...
5,Sat Oct 12 11:24:16 +0000 2024,بورتو ريمس\n\n▬نمشؾ▬\n◗CX10◖\n\n▬ٺون▬\n◗EWW◖
6,Sat Oct 12 11:24:15 +0000 2024,شالكه لاتسيو استوريل\n\n▬نمشؾ▬\n◗CX10◖\n\n▬ٺون...
7,Tue Oct 08 16:08:40 +0000 2024,"Fast smarter, feel better! 🍽️🔥 Discover the po..."
8,Sat Oct 12 11:24:10 +0000 2024,I’M IN MADRID
9,Sat Oct 12 11:24:09 +0000 2024,@galinaslesar @galiamarbella @galiafuego Eres ...


In [11]:
def clean_entries_with_dates_v2(list_of_elem):
    clean_data = []
    for element in list_of_elem:
        content = element.get('content',[])                                                                             # para extraer el contenido de 'content'
        item_content = content.get('itemContent',{})                                                                    # para extraer el contenido de 'itemContent'
        if 'tweet_results' not in item_content or 'result' not in item_content['tweet_results'] : continue              # que siga de largo con este elemento si no tiene 'tweet_result' o 'result'
        result = item_content['tweet_results']['result']                                                                # para extraer el contenido de 'tweet_results'
        if 'legacy' not in result or 'full_text' not in result['legacy']: continue                                      # que siga de largo con este elemento si no tiene 'full_text' o 'legacy'
        full_text = result['legacy']['full_text']                                                                       # para extraer el contenido de 'full_text'
        post_date = result['legacy']['created_at']
        likes = result['legacy']['favorite_count']
        clean_data.append((post_date, full_text, likes))
    return clean_data



In [13]:
aux_prueba_v2 = clean_entries_with_dates_v2(entries_api_01)
#aux_prueba_v2.to_csv('aux_prueba_v2.csv')
aux_prueba_v2 = pd.DataFrame(aux_prueba_v2)
aux_prueba_v2

,0,1,2
0,Sat Oct 12 11:24:21 +0000 2024,No hay mucha discusión \nLamine\nA Vini lo veo...,0
1,Sat Oct 12 11:24:21 +0000 2024,yes i'm still salty i can't attend music bank ...,0
2,Fri Oct 11 09:32:00 +0000 2024,Vielä ehdit! Liittyminen ja 1. treenikk 10 eur...,5
3,Sat Oct 12 11:24:20 +0000 2024,@sxftmdx fuersas nosotros hemos llegado casi 2...,0
4,Sat Oct 12 11:24:19 +0000 2024,@HolaSoyMarisa4 En cuanto empiezas a insultarm...,0
5,Sat Oct 12 11:24:16 +0000 2024,بورتو ريمس\n\n▬نمشؾ▬\n◗CX10◖\n\n▬ٺون▬\n◗EWW◖,0
6,Sat Oct 12 11:24:15 +0000 2024,شالكه لاتسيو استوريل\n\n▬نمشؾ▬\n◗CX10◖\n\n▬ٺون...,0
7,Tue Oct 08 16:08:40 +0000 2024,"Fast smarter, feel better! 🍽️🔥 Discover the po...",23
8,Sat Oct 12 11:24:10 +0000 2024,I’M IN MADRID,0
9,Sat Oct 12 11:24:09 +0000 2024,@galinaslesar @galiamarbella @galiafuego Eres ...,0


In [6]:
response_api_01.json()

{'data': {'search_by_raw_query': {'search_timeline': {'timeline': {'instructions': [{'type': 'TimelineAddEntries',
       'entries': [{'entryId': 'tweet-1841897407866667376',
         'sortIndex': '1841897417469853696',
         'content': {'entryType': 'TimelineTimelineItem',
          '__typename': 'TimelineTimelineItem',
          'itemContent': {'itemType': 'TimelineTweet',
           '__typename': 'TimelineTweet',
           'tweet_results': {'result': {'__typename': 'Tweet',
             'rest_id': '1841897407866667376',
             'core': {'user_results': {'result': {'__typename': 'User',
                'id': 'VXNlcjoxNzI3NjM0ODE3MjU1MzY2NjU2',
                'rest_id': '1727634817255366656',
                'affiliates_highlighted_label': {},
                'has_graduated_access': True,
                'is_blue_verified': False,
                'profile_image_shape': 'Circle',
                'legacy': {'can_dm': False,
                 'can_media_tag': False,
            